In [1]:
# created with love in cooperation with ASPIS
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from keras.models import Model
import os
import pandas as pd
from matplotlib import pyplot
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, plot_precision_recall_curve
from sklearn.metrics import fbeta_score, auc, confusion_matrix, classification_report, mean_absolute_error, roc_curve, roc_auc_score

In [2]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [3]:
df=pd.DataFrame(columns=['col', 'epoch', 'eval_v', 'tp', 'fp', 'fn', 'tn', 'p', 'r', 'f1', 'auc-roc', 'auc-pr', 'tss'])

In [ ]:
print("###################")

print("# settings")
directory = "full_to_eval_2"
with open('eval_2.txt', 'a') as the_file:
# nastav spravne cesty
    for subdir, dirs, files in os.walk(f"/home/jovyan/data/lightning/adrian/dip/vierka_train/training/%s"%directory):
        if len(files)>0:
            a=subdir
            f_name = subdir[subdir.index("%s/"%directory):subdir.index("/autoreg")]
            config = f_name.split('/')[1]
            for file in files:
                if 'ch-05' in file:
                    saved_model = f"/home/jovyan/data/lightning/adrian/dip/vierka_train/training/{f_name}/autoreg/%s"%file
                    # skontroluj

                    X_test = np.load(f'/home/jovyan/data/lightning/adrian/dip/vierka_train/data-preparation/{f_name}/X_test.npy',allow_pickle=True).item()
                    y_test = np.load(f'/home/jovyan/data/lightning/adrian/dip/vierka_train/data-preparation/{f_name}/y_test.npy')

                    Phi60_Sig1 = X_test["Phi60_Sig1"]

                    print("###################")
                    print("# load data")
                    # niekedy je cely rok strasne vela na vyhodnotenie 
                    # zvazit pripadne takyto split - jednotne vsetky experimenty, finalne nastavenie na celom
                    y_test = y_test[ :int(len(y_test)/16)]
                    Phi60_Sig1 = Phi60_Sig1[ :int(len(Phi60_Sig1)/16)]

                    print("###################")
                    print("# predict")
                    model = keras.models.load_model(saved_model)

                    y_pred = model.predict(Phi60_Sig1)

                    print("###################")
                    print("# roc + thresh")

                    y_true = np.where(y_test >= 0.1, 1, 0)
                    y_scores = y_pred


                    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
                    print("AUC:", roc_auc_score(y_true, y_scores))

                    optimal_idx = np.argmax(tpr - fpr)
                    optimal_threshold = thresholds[optimal_idx]

                    print("Optimal threshold value is:", optimal_threshold)
                    plot_roc_curve(fpr, tpr)

                    print("###################")
                    print("# pr curve")
                    # finalne vykreslim krajsie - ako v notebooku treshold_notebook.ipynb, len to treba stale instalovat kniznice takze zatial takto
                    precision, recall, thresholds = precision_recall_curve(y_true,y_pred)
                    # plot the roc curve for the model
                    no_skill = len(y_true[y_true==1]) / len(y_true)
                    pyplot.plot([0,1], [no_skill,no_skill], linestyle='--', label='No Skill')
                    pyplot.plot(recall, precision, marker='.', label='Logistic')
                    # axis labels
                    pyplot.xlabel('Recall')
                    pyplot.ylabel('Precision')
                    pyplot.legend()
                    # show the plot
                    pyplot.show()

                    print("###################")
                    print("# evaluation v1")
                    the_file.write("\n###################")
                    the_file.write("# evaluation v1\n")

                    # zakladne vyhodnotenie bez agregacie 

                    y_pred2 = np.where(y_pred >= optimal_threshold, 1, 0)
                    y_test2 = np.where(y_test >= 0.1, 1, 0)

                    cm = confusion_matrix(y_test2, y_pred2)

                    print("Confusion matrix: \n" + str(cm))
                    print(classification_report(y_test2, y_pred2))
                    the_file.write("Confusion matrix: \n" + str(cm))
                    the_file.write(classification_report(y_test2, y_pred2))
                    
                    print("MAE: ", str(mean_absolute_error(y_test, y_pred)), "\n")

                    print("ROC-AUC: ", str(roc_auc_score(y_test2, y_pred)))
                    roc_auc = str(roc_auc_score(y_test2, y_pred))

                    precision, recall, _ = precision_recall_curve(y_test2, y_pred)
                    print("PR-AUC: ", str(auc(recall, precision)), "\n")
                    pr_auc = str(auc(recall, precision))

                    print("F-beta macro: ", str(fbeta_score(y_test2, y_pred2, average='macro', beta=np.inf)))
                    f_beta = str(fbeta_score(y_test2, y_pred2, average='macro', beta=np.inf))

                    print("F1 macro: ", str(f1_score(y_test2, y_pred2, average='macro')), "\n")
                    f1_macro = str(f1_score(y_test2, y_pred2, average='macro'))

                    #print("F-beta weighted: ", str(fbeta_score(y_test2, y_pred2, average='weighted', beta=np.inf)))
                    #print("F1 weighted: ", str(f1_score(y_test2, y_pred2, average='weighted')))
                    df.loc[len(df)]=[config, file, '1', roc_auc, pr_auc, f_beta, f1_macro]


                    print("###################")
                    print("# evaluation v2")
                    # rolling okna kde berieme maximum - toto vyhodnotenie mozeme vo finale asi vynechat 

                    df_pred = pd.DataFrame(y_pred)
                    df_test = pd.DataFrame(y_test)

                    df_test["y_test_new"] = df_test[0].rolling(window=5, min_periods=1).max()   
                    df_pred["y_pred_new"] = df_pred[0].rolling(window=5, min_periods=1).max()   



                    y_true = np.where(df_test["y_test_new"] >= 0.1, 1, 0)
                    y_scores = y_pred


                    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
                    print("AUC:", roc_auc_score(y_true, y_scores))

                    optimal_idx = np.argmax(tpr - fpr)
                    optimal_threshold2 = thresholds[optimal_idx]

                    print("Optimal threshold value is:", optimal_threshold2)
                    plot_roc_curve(fpr, tpr)



                    #y_pred2 = df_pred["y_pred_new"] 
                    #y_test2 = df_test["y_test_new"] 
                    y_pred2 = np.where(df_pred["y_pred_new"] >= optimal_threshold2, 1, 0)
                    y_test2 = np.where(df_test["y_test_new"] >= 0.1, 1, 0)

                    cm = confusion_matrix(y_test2, y_pred2)

                    print("Confusion matrix: \n" + str(cm))
                    print(classification_report(y_test2, y_pred2))
                    print("MAE: ", str(mean_absolute_error(y_test, y_pred)), "\n")

                    print("ROC-AUC: ", str(roc_auc_score(y_test2, y_pred)))
                    precision, recall, _ = precision_recall_curve(y_test2, y_pred)
                    print("PR-AUC: ", str(auc(recall, precision)), "\n")

                    print("F-beta macro: ", str(fbeta_score(y_test2, y_pred2, average='macro', beta=np.inf)))
                    print("F1 macro: ", str(f1_score(y_test2, y_pred2, average='macro')), "\n")

                    #print("F-beta weighted: ", str(fbeta_score(y_test2, y_pred2, average='weighted', beta=np.inf)))
                    #print("F1 weighted: ", str(f1_score(y_test2, y_pred2, average='weighted')))

                    print("###################")
                    print("# evaluation v3")
                    the_file.write("###################")
                    the_file.write("# evaluation v3")
                    # aregovane vyhodnocovanie po binarizacii sa berie okno velkosti 6 v ktorom musia byt aspon 2 scintilacie
                    # - treba experimentovat s tymito cislami

                    df_pred = pd.DataFrame(y_pred)
                    df_test = pd.DataFrame(y_test)

                    pred_bin = np.where(df_pred[0] >= optimal_threshold, 1, 0)
                    test_bin  = np.where(df_test[0] >= 0.1, 1, 0)

                    # tu je moznost to menit dala som ze v okne 6 sa ma nachadzat hodnota vacsia ako treshold minimalne 2x
                    df_pred["pred_bin"] = pred_bin
                    cond = df_pred["pred_bin"].rolling(window=6, min_periods=1).sum().ge(2)  


                    df_test["test_bin"] = test_bin
                    cond2 = df_test["test_bin"].rolling(window=6, min_periods=1).sum().ge(2) 


                    df_pred["y_pred_new"] = df_pred["pred_bin"].where(cond).replace(0,1).fillna(0)
                    df_test["y_test_new"] = df_test["test_bin"].where(cond2).replace(0,1).fillna(0)


                    y_pred2 = df_pred["y_pred_new"] 
                    y_test2 = df_test["y_test_new"] 

                    cm = confusion_matrix(y_test2, y_pred2)

                    print("Confusion matrix: \n" + str(cm))
                    print(classification_report(y_test2, y_pred2))
                    the_file.write("Confusion matrix: \n" + str(cm))
                    the_file.write(classification_report(y_test2, y_pred2))
                    
                    
                    print("MAE: ", str(mean_absolute_error(y_test, y_pred)), "\n")

                    print("ROC-AUC: ", str(roc_auc_score(y_test2, y_pred)))
                    roc_auc = str(roc_auc_score(y_test2, y_pred))

                    precision, recall, _ = precision_recall_curve(y_test2, y_pred)
                    print("PR-AUC: ", str(auc(recall, precision)), "\n")
                    pr_auc = str(auc(recall, precision))

                    print("F-beta macro: ", str(fbeta_score(y_test2, y_pred2, average='macro', beta=np.inf)))
                    f_beta = str(fbeta_score(y_test2, y_pred2, average='macro', beta=np.inf))

                    print("F1 macro: ", str(f1_score(y_test2, y_pred2, average='macro')), "\n")
                    f1_macro = str(f1_score(y_test2, y_pred2, average='macro'))


                    df.loc[len(df)]=[config, file, '3', roc_auc, pr_auc, f_beta, f1_macro]

                    #print("F-beta weighted: ", str(fbeta_score(y_test2, y_pred2, average='weighted', beta=np.inf)))
                    #print("F1 weighted: ", str(f1_score(y_test2, y_pred2, average='weighted')))
                              

df.to_csv('{}.csv'.format(config))
